In [ ]:
import os
import cv2
import pandas as pd
from os import listdir
from os.path import isfile, join
from sklearn.metrics import f1_score
GT_FOLDER = 'groundTruth/'
SOLUTIONS_FOLDER = 'solutii/'

candidate_results = []
candidate_folders = [x[0] for x in os.walk(SOLUTIONS_FOLDER)]
candidate_folders.remove(SOLUTIONS_FOLDER)
for candidate_folder in candidate_folders:
    prediction_folder = candidate_folder + 'predictions/'
    pred_files = [f for f in listdir(prediction_folder) if isfile(join(prediction_folder, f))]
    gt_files = [f for f in listdir(GT_FOLDER) if isfile(join(GT_FOLDER, f))]
    candidate_scores = []
    for pred_file, gt_file in zip(pred_files, gt_files):
        pred = cv2.imread(pred_file, cv2.IMREAD_GRAYSCALE)
        gt = cv2.imread(gt_file, cv2.IMREAD_GRAYSCALE)

        if pred is None or gt is None:
            score = 0 

        elif pred.shape != gt.shape:
            score = 0
        
        else:

            # Flatten to 1D for metric calculation
            pred_flat = pred.flatten()
            gt_flat = gt.flatten()

            # Compute F1-score
            score = f1_score(gt_flat, pred_flat)

        candidate_scores.append(score)

    total_score = sum(candidate_scores)
    candidate_results.append({'Name' : candidate_folder, 'Detailed': candidate_scores, 'Total' : total_score})

df = pd.DataFrame(candidate_results)
df = df.sort_values(by="Total", ascending=False)  # Sort in descending order (highest score first)
df.to_csv("rezultate.csv", index=False)